In [30]:
import numpy as np
import torch
import torch.nn as nn
from auto_LiRPA import BoundedModule, BoundedTensor, PerturbationLpNorm
import torch.nn.functional as F

#constants
TARGET = np.array([[-0.1, 0.2, 0.3]], dtype = np.double)


EPS = 0.5

W0 = np.array([[-0.5, -0.03, -0.08],
               [ 0.15,  0.19,  0.27]], dtype = np.double)
B0 = np.array([-0.46, -0.02], dtype = np.double)

W1 = np.array([[ 0.6880, -0.4974],
               [ 0.3441, -0.6595],
               [ 0.1442, -0.0750]], dtype = np.double)
B1 = np.array([ 0.5848, -0.2861, -0.0015], dtype = np.double)


In [38]:
#a very simple neural net
class LinearModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc0 = nn.Linear(3, 2)
        self.fc0.weight = torch.nn.Parameter(torch.from_numpy(W0))
        self.fc0.bias = torch.nn.Parameter(torch.from_numpy(B0))
        
        self.fc1 = nn.Linear(2,3)
        self.fc1.weight = torch.nn.Parameter(torch.from_numpy(W1))
        self.fc1.bias = torch.nn.Parameter(torch.from_numpy(B1))
        
    def forward(self, x):
        return self.fc1(F.relu(self.fc0(x)))
        
#Compute bound using Interval Bound Propagation, using auto_LiRPA API
target = torch.from_numpy(TARGET)
                        
ball = PerturbationLpNorm(norm=np.inf, eps=EPS)
ball_tensor = BoundedTensor(target, ball)
print(ball_tensor)

original_model = LinearModel()
print(original_model.fc1.weight, original_model.fc1.bias)
lirpa_model = BoundedModule(original_model, torch.empty_like(target))

print(lirpa_model(ball_tensor))

# lb, ub = lirpa_model.compute_bounds(IBP=True, method = 'forward')
lb, ub = lirpa_model.compute_bounds(x = (ball_tensor, ), IBP=True, method = 'forward')
ibp_bound = np.array([lb.detach().numpy(), ub.detach().numpy()])
print(ibp_bound)

<BoundedTensor: tensor([[-0.1000,  0.2000,  0.3000]], dtype=torch.float64), PerturbationLpNorm(norm=inf, eps=0.5)>
Parameter containing:
tensor([[ 0.6880, -0.4974],
        [ 0.3441, -0.6595],
        [ 0.1442, -0.0750]], dtype=torch.float64, requires_grad=True) Parameter containing:
tensor([ 0.5848, -0.2861, -0.0015], dtype=torch.float64, requires_grad=True)
tensor([[ 0.5430, -0.3415, -0.0078]], dtype=torch.float64,
       grad_fn=<AddBackward0>)
[[[ 0.3913114 -0.5426455 -0.030675 ]]

 [[ 0.5848    -0.2861    -0.0015   ]]]


In [ ]:
#Compute the output of the network
def forward(x):
    fc0 = np.matmul(x, W0.transpose()) + B0
    return fc0
print(lirpa_model(ball_tensor).detach().numpy())
print(forward(TARGET))
assert(np.array_equal(forward(TARGET),
                      lirpa_model(ball_tensor).detach().numpy()))

In [43]:
##Compute bound using Interval Bound Propagation, using my own implementation.
##The closed-form solution is in eq(6), https://arxiv.org/pdf/1810.12715.pdf
my_ball = np.array([TARGET - EPS, TARGET + EPS])
print(my_ball.shape)

def my_IBP(prev_bound: np.array, W: np.array, b: np.array):
    """
    prev_bound: 2x784 prev_bound[0][i]: lower of unit ith, prev_bound[1]i]: upper of unit ith
    W: 784x256
    b: 256
    """
    assert(prev_bound.shape[0]==2)
    assert(prev_bound.shape[-1]==W.shape[0])
    assert(W.shape[1] == b.shape[0])
    
    prev_u = (prev_bound[0,:] + prev_bound[1,:])/2

    prev_r = (prev_bound[1,:] - prev_bound[0,:])/2
    
    u = np.matmul(prev_u , W) + b
    
    r = np.matmul(prev_r , abs(W) )
    
    new_bound_lower = u - r
    
    new_bound_upper = u + r
    
    new_bound = np.array([new_bound_lower, new_bound_upper])
    
    return new_bound


fc0_bound = np.maximum(my_IBP(my_ball, W0.transpose(), B0), 0)
fc1_bound = my_IBP(fc0_bound, W1.transpose(), B1)


print("my_bound:\n", fc1_bound)
print("ibp_bound:\n", ibp_bound)

#cannot use array_equal due to some floating point difference?
assert(np.allclose(fc1_bound, 
                   ibp_bound))


(2, 1, 3)
my_bound:
 [[[ 0.3913114 -0.5426455 -0.030675 ]]

 [[ 0.5848    -0.2861    -0.0015   ]]]
ibp_bound:
 [[[ 0.3913114 -0.5426455 -0.030675 ]]

 [[ 0.5848    -0.2861    -0.0015   ]]]


In [23]:
## Computer bound using CROWN
print(lirpa_model(ball_tensor))
crown_lb, crown_ub = lirpa_model.compute_bounds(IBP=False, method = 'backward')
crown_bound = np.array([crown_lb.detach().numpy(), crown_ub.detach().numpy()])
print(crown_bound)


tensor([[-0.4400,  0.0840]], dtype=torch.float64, grad_fn=<AddBackward0>)
[[[-0.745 -0.221]]

 [[-0.135  0.389]]]
